In [1]:
from functools import partial
from pyscf import gto, scf, cc
import numpy as np

from jax import config
config.update("jax_enable_x64", True)

print = partial(print, flush=True)

a = 1.831  #1.05835 # 2aB
d = a
nH = 10
atoms = ""
for i in range(nH):
    if i % 2 == 0:
        atoms += f"H {i/2*(a+d):.5f} 0.00000 0.00000 \n"
    else:
        atoms += f"H {(i-1)/2*(a+d)+a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="sto6g", unit= "bohr", verbose=4)
mol.build()

mf = scf.RHF(mol).density_fit()
e = mf.kernel()

mycc = cc.CCSD(mf)
mycc.kernel()
et = mycc.ccsd_t()


System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Sun Sep 21 13:07:50 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 10
[INPUT] num. electrons = 10
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = bohr
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.0000

In [2]:
from pyscf import ci
myci = ci.CISD(mf)
myci.kernel()


WARN: DF-RCISD for DFHF method <pyscf.df.df_jk.DFRHF object at 0x7f530dcdbf10> is not available. Normal RCISD method is called.


WARN: CISD detected DF being used in the HF object. MO integrals are computed based on the DF 3-index tensors.
It's recommended to use dfccsd.CCSD for the DF-CISD calculations


******** <class 'pyscf.ci.cisd.RCISD'> ********
CISD nocc = 5, nmo = 10
max_cycle = 50
direct = 0
conv_tol = 1e-09
max_cycle = 50
max_space = 12
lindep = 0
nroots = 1
max_memory 4000 MB (current use 172 MB)
Init t2, MP2 energy = -0.102474227433649
RCISD converged
E(RCISD) = -5.412603932738157  E_corr = -0.1454948575739534


(-0.14549485757395342,
 array([ 9.39531209e-01, -4.24476931e-16, -6.23532341e-04,  3.71833928e-16,
        -8.56553399e-04, -5.15238768e-16, -1.35554589e-03,  4.13815022e-16,
        -2.27902398e-03, -2.75066711e-16,  7.11380597e-04,  1.93953112e-15,
        -5.31576088e-03, -6.32257959e-16,  1.73047645e-03,  1.34334040e-16,
         1.05820857e-02,  5.23724141e-16, -3.60754784e-03,  3.62482638e-16,
        -4.63777009e-04,  6.85525652e-16, -6.29219928e-03,  7.07485963e-16,
        -7.78939018e-04,  6.44047119e-16, -2.37756840e-02,  1.12690547e-16,
         1.01666800e-02, -3.39204296e-18, -3.15300071e-04,  1.13748093e-16,
        -1.66723565e-02,  1.27292666e-16,  7.86174883e-03,  1.11043806e-16,
         1.01666800e-02,  1.28888362e-16, -1.43282251e-02, -5.35973558e-17,
         6.48434922e-03, -5.92696507e-18,  7.86174883e-03, -5.21527781e-17,
        -1.16754398e-02,  2.00894617e-16, -3.15300071e-04,  1.10517350e-16,
         6.48434922e-03,  2.00977030e-16, -1.59361461e-02,  1.848

In [ ]:
v_ci = myci.ci
c0,c1,c2 = myci.cisdvec_to_amplitudes(v_ci)
c1 = c1/c0
c2 = c2/c0

In [55]:
from pyscf import lib
einsum = lib.einsum

In [ ]:
# t1 = mycc.t1
# t2 = mycc.t2
# c2_t1 = lib.einsum('ia,jb->ijab',t1,t1)
# c2_t1t2 = t2 + c2_t1

In [ ]:
# print(t2[i,j,a,b])
# print(t2[j,i,b,a])

-0.03165936094408821
-0.03165936094408821


In [ ]:
# print(c2_t1[i,j,a,b])
# print(c2_t1[j,i,b,a])
# print(c2_t1t2[i,j,a,b])
# print(c2_t1t2[j,i,b,a])

1.4546614046367362e-31
1.4546614046367362e-31
-0.03165936094408821
-0.03165936094408821


In [ ]:
# c3_t1 = lib.einsum('ia,jb,kc->ijkabc',t1,t1,t1)

In [ ]:
# i = a = 0
# j = b = 1
# k = c = 2
# print(c3_t1[i,j,k,a,b,c])
# print(c3_t1[j,i,k,b,a,c])
# print(c3_t1[i,k,j,a,c,b])

-3.0442466890749704e-46
-3.04424668907497e-46
-3.0442466890749704e-46


In [ ]:
# c3_t1t2 = 1/3*(lib.einsum('ia,jkbc->ijkabc',t1,t2)
#                +lib.einsum('jb,ikac->ijkabc',t1,t2)
#                +lib.einsum('kc,jiba->ijkabc',t1,t2))

In [ ]:
# print(c3_t1t2[i,j,k,a,b,c])
# print(c3_t1t2[j,i,k,b,a,c])
# print(c3_t1t2[k,i,j,c,a,b])

4.7584814047242256e-17
4.7584814047242256e-17
4.7584814047242256e-17


In [ ]:
# c3_t1_t1t2 = lib.einsum('ia,jb,kc->ijkabc',t1,t1,t1) \
#             + (lib.einsum('ia,jkbc->ijkabc',t1,t2)
#                +lib.einsum('jb,ikac->ijkabc',t1,t2)
#                +lib.einsum('kc,jiba->ijkabc',t1,t2))

In [ ]:
# print(c3_t1_t1t2[i,j,k,a,b,c])
# print(c3_t1_t1t2[j,i,k,b,a,c])
# print(c3_t1_t1t2[k,i,j,c,a,b])

1.4275444214172678e-16
1.4275444214172678e-16
1.4275444214172678e-16


In [73]:
def t1t2_2_c1c2c3(t1,t2):
    # in closed shell convention
    # t1t2 has to be symmetrized
    c1 = t1
    c2 = t2 + lib.einsum('ia,jb->ijab',t1,t1)
    c3 = lib.einsum('ia,jb,kc->ijkabc',t1,t1,t1) \
            + (lib.einsum('ia,jkbc->ijkabc',t1,t2)
               +lib.einsum('jb,ikac->ijkabc',t1,t2)
               +lib.einsum('kc,jiba->ijkabc',t1,t2))
    return c1,c2,c3

In [ ]:
def afqmc_cr(e_afqmc,c1,c2,c3):
    o1 = 2*einsum('ia,ia->',c1,c1)
    o2 = 2*einsum('ijab,ijab->',c2,c2) - einsum('ijab,ijba->',c2,c2)
    o3 = (8*einsum('ijkabc,ijkabc->',c3,c3)-4*(einsum('ijkabc,ijkacb->',c3,c3)
        + einsum('ijkabc,ijkcba->',c3,c3) + einsum('ijkabc,ijkbac->',c3,c3))
        + 2*(einsum('ijkabc,ijkbca->',c3,c3) + einsum('ijkabc,ijkcab->',c3,c3)))/6
    cr_factor = o3/(1+o1+o2+o3)
    e_cr = (cr_factor-cr_factor**2)*e_afqmc
    return e_cr

In [82]:
t1,t2 = mycc.t1, mycc.t2
c1,c2,c3 = t1t2_2_c1c2c3(t1,t2)
o1 = 2*einsum('ia,ia->',c1,c1)
o2 = 2*einsum('ijab,ijab->',c2,c2) - einsum('ijab,ijba->',c2,c2)
o3 = (8*einsum('ijkabc,ijkabc->',c3,c3)-4*(einsum('ijkabc,ijkacb->',c3,c3)
    + einsum('ijkabc,ijkcba->',c3,c3) + einsum('ijkabc,ijkbac->',c3,c3))
    + 2*(einsum('ijkabc,ijkbca->',c3,c3) + einsum('ijkabc,ijkcab->',c3,c3)))/6

In [84]:
print(o1,o2,o3)

0.0009455421109037303 0.1640110434201618 6.647880076290393e-05


In [83]:
print(2*einsum('ijab,ijab->',c2,c2) - einsum('ijab,ijba->',c2,c2))
print(2*einsum('ijab,ijab->',c2,c2) - einsum('ijab,jiab->',c2,c2))

0.1640110434201618
0.16401104342016182


In [71]:
# a=d=1.831 eql
# converged SCF energy = -10.5113174115856
# RCCSD converged
# E(RCCSD) = -10.82386829894422  E_corr = -0.3125508873585868
# DMRG energy =  -10.832557941737981
# AFQMC energy: -10.83072 +/- 0.00009
e_afqmc = -10.83072
e_dmrg = -10.832557941737981
t1 = mycc.t1
t2 = mycc.t2
c1,c2,c3 = t1t2_2_c1c2c3(t1,t2)
e_cr = afqmc_cr(e_afqmc,c1,c2,c3)
print(e_cr,e_afqmc+e_cr)
print(e_afqmc+e_cr-e_dmrg)

-0.004481373160699468 -10.835201373160698
-0.0026434314227170574


In [75]:
# a=1.831 d=3
# converged SCF energy = -10.7902563668634
# E_corr(MP2) -0.196801273811774
# E(RCCSD) = -11.11588133074048  E_corr = -0.3256249638770554
# DMRG energy (extrapolated) =  -11.116780958599403 +/- 0.0000000000
# AFQMC energy: -11.11581 +/- 0.00009
e_afqmc = -11.11581
e_dmrg = -11.116780958599403
t1 = mycc.t1
t2 = mycc.t2
c1,c2,c3 = t1t2_2_c1c2c3(t1,t2)
e_cr = afqmc_cr(e_afqmc,c1,c2,c3)
print(e_cr,e_afqmc+e_cr)
print(e_afqmc+e_cr-e_dmrg)

-0.0009076929412274924 -11.116717692941227
6.32656581753821e-05


In [77]:
# H40 a=d=1.831
# converged SCF energy = -21.0019645737223
# E(RCCSD) = -21.62210834629252  E_corr = -0.6201437725701879
# RCCSD(T) correction = -0.0176994487844547
# ccsd(t) energy: -21.639807795076976
# AFQMC energy: -21.6382 +/- 0.0004
# DMRG energy (extrapolated) =  -21.646735579929590 +/-    0.0000000000
e_afqmc = -21.6382
e_dmrg = -21.646735579929590
t1 = mycc.t1
t2 = mycc.t2
c1,c2,c3 = t1t2_2_c1c2c3(t1,t2)
e_cr = afqmc_cr(e_afqmc,c1,c2,c3)
print(e_cr,e_afqmc+e_cr)
print(e_afqmc+e_cr-e_dmrg)

-0.04139135783301887 -21.67959135783302
-0.03285577790342842


In [ ]:
# H40 a=1.831 d=3
# converged SCF energy = -21.5756025423033
# E(RCCSD) = -22.22585047306383  E_corr = -0.6502479307605629
# RCCSD(T) correction = -0.0010780749331425
# ccsd(t) energy: -22.226928547996973
# AFQMC energy: -22.2245 +/- 0.0004
# DMRG energy (extrapolated) =  -22.227757880107031 +/-    0.0000000000
e_afqmc = -22.2245
e_dmrg = -22.227757880107031
t1 = mycc.t1
t2 = mycc.t2
c1,c2,c3 = t1t2_2_c1c2c3(t1,t2)
e_cr = afqmc_cr(e_afqmc,c1,c2,c3)
print(e_cr,e_afqmc+e_cr)
print(e_afqmc+e_cr-e_dmrg)

-0.007088406992146511 -22.231588406992145
-0.00028632338904088783


In [4]:
e_afqmc = -22.2245
e_dmrg = -22.227757880107031
c1,c2,c3 = t1t2_2_c1c2c3(mycc)
c = cr_factor(c1,c2,c3)
e_afqmc_cr,_ = afqmc_cisd_cr(e_afqmc,c)
print(f'err to DMRG before = {e_afqmc-e_dmrg}, after = {e_afqmc_cr-e_dmrg}')

err to DMRG before = 0.003257880107032207, after = 0.002019729674799464


In [ ]:
# H80 a = 0.96892 # 1.831aB
# AFQMC energy: -43.2513 +/- 0.0008
# converged SCF energy = -41.9835706086777
# E(RCCSD) = -43.2187515626576
# ccsd(t) energy: -43.25881794557464
# DMRG energy =  -43.277423271142652

In [116]:
e_afqmc = -43.2513
e_dmrg = -43.277423271142652
c1,c2,c3 = t1t2_2_c1c2c3(mycc)
c = cr_factor(c1,c2,c3)
e_afqmc_cr,_ = afqmc_cisd_cr(e_afqmc,c)
print(f'err to DMRG before = {e_afqmc-e_dmrg}, after = {e_afqmc_cr-e_dmrg}')

: 

In [6]:
def davidson_cr(ci_corr,c0,nelec,order=1):
    c2_int_sq = (1-c0**2)/c0**2
    d_cr = c2_int_sq*ci_corr*(1-2/nelec)
    if order == 2:
        d_cr2 = 2*ci_corr*(4/nelec**2-1)*c2_int_sq**2
        d_cr += d_cr2
    return d_cr

In [104]:
nelec = mol.nelectron
c2_int_sq = (1-c0**2)/c0**2
d_cr2 = 2*ci_corr*(4/nelec**2-1)*c2_int_sq**2

In [18]:
nelec = mol.nelectron
d_cr1 = davidson_cr(ci_corr,c0,nelec,1)
d_cr2 = davidson_cr(ci_corr,c0,nelec,2)
print(d_cr1)
print(d_cr2)

-0.04645142872131443
-0.027028322614441352


In [20]:
print(ci_corr)
print(ci_corr+d_cr1)
print(ci_corr+d_cr2)
print(mycc.e_corr+et)

-0.2715561490087084
-0.3180075777300228
-0.2985844716231497
-0.3261313621110479
